In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing import image_dataset_from_directory
import numpy as np


In [2]:

import os
import numpy as np
from tqdm import tqdm

In [3]:
IMAGE_SIZE = (224, 224)
DIMENSION_EMBEDDING = 256
BATCH_SIZE = 32

DATASET_IMAGE_POKEMON_DIRECTORY = "../data/pokemon-dataset-1000"
OUTPUT_IMAGE_POKEMON_EMBEDDE_DIRECTORY = "../data/dataset_embedding"



In [4]:
def loader_dataset_finetuner():
    model_finetune = load_model("../models/finetuned_efficientnetb0_pour_pokemon.h5")

    model_finetune = Model(inputs=model_finetune.input, outputs=model_finetune.get_layer("embedding").output)

    return model_finetune

In [5]:
model_finetune = loader_dataset_finetuner()

dataset = image_dataset_from_directory(
    DATASET_IMAGE_POKEMON_DIRECTORY, 
    image_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    shuffle=False
)

Found 20921 files belonging to 1000 classes.


In [6]:
class_names = dataset.class_names

In [ ]:
#pour accelerer le pretraitement des images
dataset = dataset.map(
    lambda x, y: preprocess_input(x),
    num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

In [ ]:
image_infos = []

for class_name in class_names:
    classe_directory = os.path.join(DATASET_IMAGE_POKEMON_DIRECTORY, class_name)
    for filename in sorted(os.listdir(classe_directory)):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_infos.append((class_name, filename))

In [ ]:
index = 0

for batch in tqdm(dataset):
    embeddings = model_finetune.predict(batch, verbose=0)
    embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

    for embedding in embeddings:
        pokemon, filename = image_infos[index]
        os.makedirs(os.path.join(OUTPUT_IMAGE_POKEMON_EMBEDDE_DIRECTORY, pokemon), exist_ok=True)
        os.makedirs(OUTPUT_IMAGE_POKEMON_EMBEDDE_DIRECTORY, exist_ok=True)

        output_path = os.path.join(OUTPUT_IMAGE_POKEMON_EMBEDDE_DIRECTORY,pokemon, filename.rsplit('.', 1)[0] + '.npy')

        np.save(output_path, embedding)

        index += 1

  8%|▊         | 53/654 [00:18<03:31,  2.84it/s]